<div style="width:100%; height:140px">
    <img src="https://www.kuleuven.be/internationaal/thinktank/fotos-en-logos/ku-leuven-logo.png/image_preview" width = 300px, heigh = auto align=left>
</div>


KUL H02A5a Computer Vision: Group Assignment 1
---------------------------------------------------------------
Student numbers: <span style="color:red">r0611202, r0633222, r3, r4, r5</span>. (fill in your student numbers!)

In this group assignment your team will delve into some deep learning applications for computer vision. The assignment will be delivered in the same groups from *Group assignment 0* and you start from this template notebook. The notebook you submit for grading is the last notebook you submit in the [Kaggle competition](https://www.kaggle.com/t/b7a2a8743bd842ca9ac93ae91cbc8d9f) prior to the deadline on **Tuesday 18 May 23:59**. Closely follow [these instructions](https://github.com/gourie/kaggle_inclass) for joining the competition, sharing your notebook with the TAs and making a valid notebook submission to the competition. A notebook submission not only produces a *submission.csv* file that is used to calculate your competition score, it also runs the entire notebook and saves its output as if it were a report. This way it becomes an all-in-one-place document for the TAs to review. As such, please make sure that your final submission notebook is self-contained and fully documented (e.g. provide strong arguments for the design choices that you make). Most likely, this notebook format is not appropriate to run all your experiments at submission time (e.g. the training of CNNs is a memory hungry and time consuming process; due to limited Kaggle resources). It can be a good idea to distribute your code otherwise and only summarize your findings, together with your final predictions, in the submission notebook. For example, you can substitute experiments with some text and figures that you have produced "offline" (e.g. learning curves and results on your internal validation set or even the test set for different architectures, pre-processing pipelines, etc). We advise you to first go through the PDF of this assignment entirely before you really start. Then, it can be a good idea to go through this notebook and use it as your first notebook submission to the competition. You can make use of the *Group assignment 1* forum/discussion board on Toledo if you have any questions. Good luck and have fun!

---------------------------------------------------------------
NOTES:
* This notebook is just a template. Please keep the five main sections, but feel free to adjust further in any way you please!
* Clearly indicate the improvements that you make! You can for instance use subsections like: *3.1. Improvement: applying loss function f instead of g*.


# 1. Overview
This assignment consists of *three main parts* for which we expect you to provide code and extensive documentation in the notebook:
* Image classification (Sect. 2)
* Semantic segmentation (Sect. 3)
* Adversarial attacks (Sect. 4)

In the first part, you will train an end-to-end neural network for image classification. In the second part, you will do the same for semantic segmentation. For these two tasks we expect you to put a significant effort into optimizing performance and as such competing with fellow students via the Kaggle competition. In the third part, you will try to find and exploit the weaknesses of your classification and/or segmentation network. For the latter there is no competition format, but we do expect you to put significant effort in achieving good performance on the self-posed goal for that part. Finally, we ask you to reflect and produce an overall discussion with links to the lectures and "real world" computer vision (Sect. 5). It is important to note that only a small part of the grade will reflect the actual performance of your networks. However, we do expect all things to work! In general, we will evaluate the correctness of your approach and your understanding of what you have done that you demonstrate in the descriptions and discussions in the final notebook.

## 1.1 Deep learning resources
If you did not yet explore this in *Group assignment 0 (Sect. 2)*, we recommend using the TensorFlow and/or Keras library for building deep learning models. You can find a nice crash course [here](https://colab.research.google.com/drive/1UCJt8EYjlzCs1H1d1X0iDGYJsHKwu-NO).

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os 
import multiprocessing
import wandb
# !pip install wandb -qqq
from wandb.keras import WandbCallback
import cv2
from ipywidgets import fixed, interact 
import ipywidgets
from albumentations import (
    Compose, HorizontalFlip, CLAHE, HueSaturationValue,
    RandomBrightness, RandomContrast, RandomGamma,
    ToFloat, ShiftScaleRotate, RandomBrightnessContrast, RandomCrop)


In [ ]:
import sys
sys.executable
sys.path

## 1.2 PASCAL VOC 2009
For this project you will be using the [PASCAL VOC 2009](http://host.robots.ox.ac.uk/pascal/VOC/voc2009/index.html) dataset. This dataset consists of colour images of various scenes with different object classes (e.g. animal: *bird, cat, ...*; vehicle: *aeroplane, bicycle, ...*), totalling 20 classes.

In [ ]:
# Loading the training data
train_df = pd.read_csv('data/train/train_set.csv', index_col="Id")
labels = train_df.columns
train_df["img"] = [np.load('data/train/img/train_{}.npy'.format(idx)) for idx, _ in train_df.iterrows()]
train_df["seg"] = [np.load('data/train/seg/train_{}.npy'.format(idx)) for idx, _ in train_df.iterrows()]
print("The training set contains {} examples.".format(len(train_df)))

# Show some examples
fig, axs = plt.subplots(2, 20, figsize=(10 * 20, 10 * 2))
for i, label in enumerate(labels):
    df = train_df.loc[train_df[label] == 1]
    axs[0, i].imshow(df.iloc[0]["img"], vmin=0, vmax=255)
    axs[0, i].set_title("\n".join(label for label in labels if df.iloc[0][label] == 1), fontsize=40)
    axs[0, i].axis("off")
    axs[1, i].imshow(df.iloc[0]["seg"], vmin=0, vmax=20)  # with the absolute color scale it will be clear that the arrays in the "seg" column are label maps (labels in [0, 20])
    axs[1, i].axis("off")
    
plt.show()

# The training dataframe contains for each image 20 columns with the ground truth classification labels and 20 column with the ground truth segmentation maps for each class
train_df.head(1)

In [ ]:
# Loading the test data
test_df = pd.read_csv('data/test/test_set.csv', index_col="Id")
test_df["img"] = [np.load('data/test/img/test_{}.npy'.format(idx)) for idx, _ in test_df.iterrows()]
test_df["seg"] = [-1 * np.ones(img.shape[:2], dtype=np.int8) for img in test_df["img"]]
print("The test set contains {} examples.".format(len(test_df)))

# The test dataframe is similar to the training dataframe, but here the values are -1 --> your task is to fill in these as good as possible in Sect. 2 and Sect. 3; in Sect. 6 this dataframe is automatically transformed in the submission CSV!
test_df.head(1)

## 1.3 Your Kaggle submission
Your filled test dataframe (during Sect. 2 and Sect. 3) must be converted to a submission.csv with two rows per example (one for classification and one for segmentation) and with only a single prediction column (the multi-class/label predictions running length encoded). You don't need to edit this section. Just make sure to call this function at the right position in this notebook.

In [ ]:
def _rle_encode(img):
    """
    Kaggle requires RLE encoded predictions for computation of the Dice score (https://www.kaggle.com/lifa08/run-length-encode-and-decode)

    Parameters
    ----------
    img: np.ndarray - binary img array
    
    Returns
    -------
    rle: String - running length encoded version of img
    """
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    rle = ' '.join(str(x) for x in runs)
    return rle

def generate_submission(df):
    """
    Make sure to call this function once after you completed Sect. 2 and Sect. 3! It transforms and writes your test dataframe into a submission.csv file.
    
    Parameters
    ----------
    df: pd.DataFrame - filled dataframe that needs to be converted
    
    Returns
    -------
    submission_df: pd.DataFrame - df in submission format.
    """
    df_dict = {"Id": [], "Predicted": []}
    for idx, _ in df.iterrows():
        df_dict["Id"].append(f"{idx}_classification")
        df_dict["Predicted"].append(_rle_encode(np.array(df.loc[idx, labels])))
        df_dict["Id"].append(f"{idx}_segmentation")
        df_dict["Predicted"].append(_rle_encode(np.array([df.loc[idx, "seg"] == j + 1 for j in range(len(labels))])))
    
    submission_df = pd.DataFrame(data=df_dict, dtype=str).set_index("Id")
    submission_df.to_csv("submission.csv")
    return submission_df

# Dataset class for classification models

In [ ]:
class Dataset_Classification(object): 
    """
     A dataset class usefull when training a classification model. 
    """
    def __init__(self, config): 
        self.config = config
        
        # set labels in a usefull format
        self._classification_labels()
        
        # initialize
        self.initialize()
        
        # to keep track of sampling
        self.sampling_check = np.zeros(20)
        self.times_sampled = 0
        
        
    def initialize(self):
        """
            Performs necessary thingss
        """
        self.train_data_path = 'data/train/img'
        self.test_data_path = 'data/test/img'
        
        # count nbr of files within data set. 
        self.nbr_of_train_images = len(os.listdir(self.train_data_path))
        self.nbr_of_test_images = len(os.listdir(self.test_data_path))
        
        # prepare train/validation split 
        train_fraction = self.config['train_fraction']
        r_idx=np.random.permutation(self.nbr_of_train_images)
        
        self.train_indices = r_idx[:int(train_fraction*self.nbr_of_train_images)]
        self.train_sample_probs = self.probabilities[self.train_indices]/np.sum(self.probabilities[self.train_indices])
        
        self.validation_indices = r_idx[int(train_fraction*self.nbr_of_train_images):]
        self.validation_sample_probs = self.probabilities[self.validation_indices]/np.sum(self.probabilities[self.validation_indices])
        
        print('Found {} train images'.format(self.nbr_of_train_images))
        print('- {} used for training, {} used for validating'.format(len(self.train_indices), len(self.validation_indices)))
        print('Found {} test images'.format(self.nbr_of_test_images))
        
        if self.config['augmentation']: 
            print('Including augmentation when training data is generated')
        self.augment = Compose([
                        #RandomCrop(width=self.config['input_shape'][0], height=self.config['input_shape'][0]),
                        HorizontalFlip(p=0.5),
                        RandomContrast(limit=0.1,p=0.25),
                        #RandomGamma(gamma_limit=(80, 120), p=0.5),
                        RandomBrightness(limit=0.15, p=0.5),
                        RandomBrightnessContrast(p=0.2),
                        HueSaturationValue(hue_shift_limit=1.5, sat_shift_limit=5,
                                           val_shift_limit=2.5, p=.7),
                        # CLAHE(p=1.0, clip_limit=2.0),
                        ShiftScaleRotate(
                            shift_limit=0.1, scale_limit=0.1, 
                            rotate_limit=15, border_mode=cv2.BORDER_REFLECT_101, p=0.8), 
                    ])
    
        self.preprocessor = lambda x: x #defualt preprocessor does nothing 
        
    def get_test_set_size(self): 
        return self.nbr_of_test_images
    def get_train_set_size(self): 
        return self.nbr_of_train_images
        
    def reshape(self,im): 
        return cv2.resize(im, self.config['input_shape'])
        
    def _classification_labels(self): 
        """
            Get the classification labels 
        """
        # label names 
        train_df = pd.read_csv('data/train/train_set.csv', index_col="Id")
        self.label_names = train_df.columns.to_numpy()
        
        # get rid of pandas frame 
        self.labels = train_df.to_numpy() # each row corresponds to label
        
        # instances per class
        absolute_nbr_of_instances_per_class = np.sum(self.labels,axis=0)
        # get array with label name for each image 
        self.class_name_per_image = list()
        probabilities = list()
        total_class_prob = 1/20*np.ones(20)
        #total_class_prob[14]=0
        for row in self.labels: #can probably be done more eligant
            idx = np.where(row==1)[0]
            if 14 in idx:
                idx=14
            else:
                idx=idx[0]
            probabilities.append(total_class_prob[idx]/absolute_nbr_of_instances_per_class[idx])
            self.class_name_per_image.append(self.label_names[idx])
        
        
        # make sure total probability sums to 1
        if self.config['uniform_sample_probabilities']:
            self.probabilities = np.ones(np.array(probabilities).shape)/len(probabilities) # uniform sampling
        else:
            self.probabilities = np.array(probabilities)/np.sum(probabilities) # sampling based on distribution of classes in trainning data
            
            
    def feed_preprocess_function(self, preprocessor): 
        """
            Each network needs it's batches preprocessed in some manner. Feed this function to the Dataset object 
            who will call it when asking for batches.
            
            The preprocessor takes 
        """
        self.preprocessor = preprocessor
    
    def prepare_image(self, image): 
        """
            Function that performs all necessary steps from input image to image passed during trainnig. 
            This method should be overwritten depending on the model used.
        """
        h,w,c=image.shape
        # resize manually, when augmentation is turned on a random crop will be done 100% of times.
        #if not self.config['augmentation'] or h < self.config['input_shape'][0]or w < self.config['input_shape'][1]:
        image = self.reshape(image)

        # augment if augmentation is turned on 
        if self.config['augmentation']:
#             print('Augmentation enabled: check if combination of preprocessor and augmentation makes sence')
            image=self.augment(image=image)["image"]
        
        # preprocess 
        image = self.preprocessor(image)
        return image
    
    def prepare_test_image(self, image): 
        """
             Same as prepare_image but without augmentation
        """
        h,w,c=image.shape
        # resize manually, when augmentation is turned on a random crop will be done 100% of times.
        #if not self.config['augmentation'] or h < self.config['input_shape'][0]or w < self.config['input_shape'][1]:
        image = self.reshape(image)
        
        # preprocess 
        image = self.preprocessor(image)
        return image
    
    def view_preprocessed_image(self, image_id, option='train'): 
        """
            Shows an image as it is passed during training/testing of the network. 
            image_id 
        """
        assert hasattr(self, 'preprocessor'), 'set a preprocessor function before using this.'
        
        # get image 
        if option=='train':
            # get image 
            real_image = np.load('data/train/img/train_{}.npy'.format(image_id) )
            label = self.class_name_per_image[image_id]
        else: 
            real_image = np.load('data/test/img/test_{}.npy'.format(image_id) )
            label = 'unknown'
            
        image=np.copy(real_image)
        
        image = self.prepare_image(image)
        
    
    
        # print some info 
        print('original image:')
        print('-original_shape:', real_image.shape)
        print('-dtype:', real_image.dtype)
        print('-min value:', np.min(real_image))
        print('-max value:', np.max(real_image))
        
        print('final image:')
        print('-final shape:', image.shape)
        print('-dtype:', image.dtype)
        print('-min value:', np.min(image))
        print('-max value:', np.max(image))
        
        
        # show figure 
        fig, axes = plt.subplots(1,2, figsize=(30,15))
        axes[0].imshow(real_image)
        axes[0].set_title('Original image', fontsize=50)
        
        axes[1].imshow(image) # clip it to [0,1] range
        axes[1].set_title('preprocessed_image', fontsize=50)
        
        plt.suptitle('label: {}'.format(label), fontsize=50)
        fig.show()
        
    def view_possible_augmentations(self, image_id): 
        fig, axes = plt.subplots(4,4, figsize=(60,30))
        real_image = np.load('data/train/img/train_{}.npy'.format(image_id) )
        plt.suptitle('original image is on the top left', fontsize=50)
        for ax in axes.flat: 
            ax.imshow(self.augment(image=real_image)["image"])
            ax.axis('off')
        axes[0,0] = plt.imshow(real_image)
        fig.show()
        
        
    
    def train_generator(self,batch_size):
        """
            generator that will feed training batches during training 
        """
        inputs = []
        targets = []
        batchcount = 0
        while True:
#             for image_id in self.train_indices:
            image_id = np.random.choice(self.train_indices, p=self.train_sample_probs)
            # sample real image
            real_image = np.load('data/train/img/train_{}.npy'.format(image_id) )

            image = self.prepare_image(np.copy(real_image))
            inputs.append(image)

            # get corresponding label 
            targets.append(self.labels[image_id])
            
            self.sampling_check+=self.labels[image_id]
            self.times_sampled+=1
            

            batchcount += 1
            if batchcount > batch_size:
                X = np.array(inputs)
                y = np.array(targets, dtype=np.uint8)
                yield (X, y)
                inputs = []
                targets = []
                batchcount = 0

    def validation_generator(self,batch_size):
        """
            generator that will feed validation batches during training 
        """
        inputs = []
        targets = []
        batchcount = 0
        while True:
            #for image_id in self.validation_indices:
            # sample real image
            image_id = np.random.choice(self.validation_indices,p=self.validation_sample_probs)
            real_image = np.load('data/train/img/train_{}.npy'.format(image_id) )

            image = self.prepare_test_image(np.copy(real_image))

            inputs.append(image)

            # get corresponding label 
            targets.append(self.labels[image_id])

            batchcount += 1
            if batchcount > batch_size:
                X = np.array(inputs)
                y = np.array(targets, dtype=np.uint8)
                yield (X, y)
                inputs = []
                targets = []
                batchcount = 0           
    
    def show_class_distribution(self): 
        fig,axes=plt.subplots(figsize=(30,15))
        class_probs=np.mean(self.labels, axis=0)
        axes.bar(self.label_names,  class_probs)
        axes.tick_params(axis='both', which='major', labelsize=30)
        for tick in axes.xaxis.get_major_ticks():
            tick.label.set_rotation('vertical')
        plt.suptitle('Class distribution within the training data.', fontsize=50)
        fig.show()
        
    def show_training_sampling_distribution(self): 
        fig,axes=plt.subplots(figsize=(30,15))
        class_probs=np.mean(self.labels, axis=0)
        axes.bar(self.label_names,  self.sampling_check/self.times_sampled)
        axes.tick_params(axis='both', which='major', labelsize=30)
        for tick in axes.xaxis.get_major_ticks():
            tick.label.set_rotation('vertical')
        plt.suptitle('Number of times each class was sampled during training.', fontsize=50)
        fig.show()
    
    def get_class_distribution(self):
        return np.mean(self.labels, axis=0)

## Visualizing data generated by Classification Dataset 

In [ ]:
dataset_config = {
    'train_fraction': 0.9,
    'input_shape': (224, 224),
    'augmentation': True, 
    'uniform_sample_probabilities': False
}
ds = Dataset_Classification(dataset_config)
ds.show_class_distribution()

In [ ]:
# resnet50 preprocessor 
ds.feed_preprocess_function(tf.keras.applications.resnet_v2.preprocess_input)


In [ ]:
image_id = ipywidgets.IntText(min=0, max=ds.get_train_set_size(), value=0, label='image_id')
option = ipywidgets.Dropdown(
    options=['train', 'test'],
    value='train',
    description='train or test:',
    disabled=False,
)
interact(ds.view_preprocessed_image,  image_id=image_id, option=option)

In [ ]:
image_id = ipywidgets.IntText(min=0, max=ds.get_train_set_size(), value=0, label='image_id')
interact(ds.view_possible_augmentations,  image_id=image_id)


# Dataset class for segmentation models

In [ ]:
class Dataset_Segmentation(Dataset_Classification): 
    """
     A dataset class usefull when training a classification model. 
    """
    def __init__(self, config): 
        self.config = config
    
        
        # set labels in a usefull format
        self._classification_labels()
        
         # initialize
        self.initialize()
      
    def prepare_image(self, image, mask): 
        """
            Function that performs all necessary steps from input image to image passed during trainnig. 
            This method should be overwritten depending on the model used.
        """
        h,w,c=image.shape
        # resize manually, when augmentation is turned on a random crop will be done 100% of times.
        #if not self.config['augmentation'] or h < self.config['input_shape'][0]or w < self.config['input_shape'][1]:
        image = self.reshape(image)
        mask=self.reshape(mask)

        # augment if augmentation is turned on 
        if self.config['augmentation']:
#             print('Augmentation enabled: check if combination of preprocessor and augmentation makes sence')
            transformed=self.augment(image=image, mask=mask)
            image=transformed['image']
            mask=transformed['mask']
        # preprocess 
        image = self.preprocessor(image)
        return image, mask
    

    def prepare_test_image(self, image, mask): 
        """
             Same as prepare_image but without augmentation
        """
        h,w,c=image.shape
        # resize manually, when augmentation is turned on a random crop will be done 100% of times.
        #if not self.config['augmentation'] or h < self.config['input_shape'][0]or w < self.config['input_shape'][1]:
        image = self.reshape(image)
        mask=self.reshape(mask)
        
        # preprocess 
        image = self.preprocessor(image)
        return image, mask 
    
    def view_preprocessed_image(self, image_id, option='train'): 
        """
            Shows the original, preprocessed and final mask used for training.
        """
        assert hasattr(self, 'preprocessor'), 'set a preprocessor function before using this.'
        
        # get image 
        if option=='train':
            # get image 
            real_image = np.load('data/train/img/train_{}.npy'.format(image_id) )
            mask = np.load('data/train/seg/train_{}.npy'.format(image_id))
            label_name=self.class_name_per_image[image_id]
        else: 
            real_image = np.load('data/test/img/test_{}.npy'.format(image_id) )
            mask = np.load('data/test/seg/test_{}.npy'.format(image_id))
            label_name='Not labelled'
        image=np.copy(real_image)
        
        image, mask = self.prepare_image(image, mask)
        
    
    
        # print some info 
        print('original image:')
        print('-original_shape:', real_image.shape)
        print('-dtype:', real_image.dtype)
        print('-min value:', np.min(real_image))
        print('-max value:', np.max(real_image))
        
        print('final image:')
        print('-final shape:', image.shape)
        print('-dtype:', image.dtype)
        print('-min value:', np.min(image))
        print('-max value:', np.max(image))
        
        
        # show figure 
        fig, axes = plt.subplots(1,3, figsize=(30,15))
        axes[0].imshow(real_image)
        axes[0].set_title('Original image', fontsize=50)
        
        axes[1].imshow(image) # clip it to [0,1] range
        axes[1].set_title('preprocessed_image', fontsize=50)
        
        axes[2].imshow(mask) # clip it to [0,1] range
        axes[2].set_title('Segmentation mask', fontsize=50)
        
        plt.suptitle('label: {}'.format(label_name), fontsize=50)
        fig.show()
        
    def view_possible_augmentations(self, image_id): 
        fig, axes = plt.subplots(4,4, figsize=(60,30))
        real_image = np.load('data/train/img/train_{}.npy'.format(image_id) )
        mask = np.load('data/train/seg/train_{}.npy'.format(image_id))
            
        plt.suptitle('original image is on the bottom right', fontsize=50)
        for ax in axes.flat: 
            transformed = self.augment(image=real_image, mask=mask)
            new_image=transformed['image']
            new_mask=transformed['mask']
            ax.imshow(new_image, alpha=1)
            ax.imshow(new_mask, alpha=0.2)
            ax.axis('off')
        axes[0,0] = plt.imshow(real_image)
        fig.show()
        
        
    
    def train_generator(self,batch_size):
        """
            generator that will feed training batches during training 
        """
        inputs = []
        targets = []
        batchcount = 0
        while True:
            for image_id in self.train_indices:
                # sample real image
                real_image = np.load('data/train/img/train_{}.npy'.format(image_id))
                mask = np.load('data/train/seg/train_{}.npy'.format(image_id))
                
                image, segmask = self.prepare_image(real_image, mask)
                inputs.append(image)
                targets.append(segmask)

                batchcount += 1
                if batchcount > batch_size:
                    X = np.array(inputs)
                    y = np.array(targets)
                    yield (X, y)
                    inputs = []
                    targets = []
                    batchcount = 0

    def validation_generator(self,batch_size):
        """
            generator that will feed validation batches during training 
        """
        inputs = []
        targets = []
        batchcount = 0
        while True:
            for image_id in self.validation_indices:
                # sample real image
                real_image = np.load('data/train/img/train_{}.npy'.format(image_id))
                mask = np.load('data/train/seg/train_{}.npy'.format(image_id) )
                
                image, segmask = self.prepare_test_image(real_image, mask)
                inputs.append(image)
                targets.append(segmask)

                batchcount += 1
                if batchcount > batch_size:
                    X = np.array(inputs)
                    y = np.array(targets)
                    yield (X, y)
                    inputs = []
                    targets = []
                    batchcount = 0           
    
    def show_class_distribution(self): 
        fig,axes=plt.subplots(figsize=(30,15))
        class_probs=np.mean(self.labels, axis=0)
        axes.bar(self.label_names,  class_probs)
        axes.tick_params(axis='both', which='major', labelsize=30)
        for tick in axes.xaxis.get_major_ticks():
            tick.label.set_rotation('vertical')
        plt.suptitle('Class distribution within the training data.', fontsize=50)
        fig.show()
    
    def get_class_distribution(self):
        return np.mean(self.labels, axis=0)

## Visualizing data generated by Segmentation Dataset 

In [ ]:
dataset_config = {
    'train_fraction': 0.9,
    'input_shape': (224, 224),
    'augmentation': True, 
    'uniform_sample_probabilities':True
}
ds = Dataset_Segmentation(dataset_config)

ds.feed_preprocess_function(lambda x: x/255.0) 

In [ ]:
image_id = ipywidgets.IntText(min=0, max=ds.get_train_set_size(), value=0, label='image_id')
option = ipywidgets.Dropdown(
    options=['train', 'test'],
    value='train',
    description='train or test:',
    disabled=False,
)
interact(ds.view_preprocessed_image,  image_id=image_id, option=option)

In [ ]:
image_id = ipywidgets.IntText(min=0, max=ds.get_train_set_size(), value=0, label='image_id')
interact(ds.view_possible_augmentations,  image_id=image_id)

# Image classification
The goal here is simple: implement a classification CNN and train it to recognise all 20 classes (and/or background) using the training set and compete on the test set (by filling in the classification columns in the test dataframe).

In [ ]:
class RandomClassificationModel:
    """
    Random classification model: 
        - generates random labels for the inputs based on the class distribution observed during training
        - assumes an input can have multiple labels
    """
    def fit(self, X, y):
        """
        Adjusts the class ratio variable to the one observed in y. 

        Parameters
        ----------
        X: list of arrays - n x (height x width x 3)
        y: list of arrays - n x (nb_classes)

        Returns
        -------
        self
        """
        self.distribution = np.mean(y, axis=0)
        print("Setting class distribution to:\n{}".format("\n".join(f"{label}: {p}" for label, p in zip(labels, self.distribution))))
        return self
        
    def predict(self, X):
        """
        Predicts for each input a label.
        
        Parameters
        ----------
        X: list of arrays - n x (height x width x 3)
            
        Returns
        -------
        y_pred: list of arrays - n x (nb_classes)
        """
        np.random.seed(0)
        return [np.array([int(np.random.rand() < p) for p in self.distribution]) for _ in X]
    
    def __call__(self, X):
        return self.predict(X)
    
model = RandomClassificationModel()
model.fit(train_df["img"], train_df[labels])
test_df.loc[:, labels] = model.predict(test_df["img"])
test_df.head(1)

## Resnet50 as a first try for classification 

In [ ]:
from tensorflow import keras
from keras import backend as K

class ResNetClassifactionModel(RandomClassificationModel): 
    def __init__(self, config): 
        self.config = config 
        self.config_head = config['head_model']
        self.project_name = 'resnet_50'
        
        # initialize dataset
        self.dataset = Dataset_Classification(config['dataset'])
        
        # feed preprocessor to dataset
        print('Feeding resnet50 preprocess function to dataset class')
        self.dataset.feed_preprocess_function(tf.keras.applications.resnet_v2.preprocess_input)
        
        # check if some configurations make sense 
        assert len(self.config_head['head_model_units']) == len(self.config_head['add_dropout']), 'head_models_units and add_dropout list should have same size'
    
    
    def set_config(self, config):
        self.config = config 
        self.config_head = config['head_model']
        self.dataset = Dataset_Classification(config['dataset'])
        assert len(self.config_head['head_model_units']) == len(self.config_head['add_dropout']), 'head_models_units and add_dropout list should have same size'
        
    def predict(self, X):
        # 
        
        if len(X.shape) == 1: 
            # X is a batch of images prepare all of them and create batch. 
            batch = np.array([self.dataset.prepare_test_image(im) for im in X])
            print(batch.shape)
            
            y = model.predict(batch)
        else: 
            # X is a single image 
            batch = self.dataset.prepare_test_image(X)
            batch = np.expand_dims(batch, axis=0)
            print(batch.shape)
            y = model.predict(batch)
            
        y = np.squeeze(y)
        print(y)
        print(self.dataset.label_names)
        label_idx = np.where(y==1.)
        print(label_idx)
        return self.dataset.label_names[label_idx]
            
            
        
        
        
        
    def build(self): 
        """
            Builds the model 
        """
        # define a resnet50 base model
        resnet50 = tf.keras.applications.ResNet50V2(
                    include_top=False,
                    weights=self.config['weights'],
                    input_shape=self.config['input_shape'],
                        )
    
        self.base_model = resnet50
        
        # define a head model
        head_model=resnet50.output
        head_model=tf.keras.layers.AveragePooling2D(pool_size=(5,5))(head_model)
        head_model=tf.keras.layers.Flatten()(head_model)
        
        for (nbr_units, dropout) in zip(self.config_head['head_model_units'], self.config_head['add_dropout']): 
            head_model=tf.keras.layers.Dense(nbr_units, activation=self.config_head['activation'])(head_model)
            if dropout:
                head_model=tf.keras.layers.Dropout(0.4)(head_model)
        
        head_model=tf.keras.layers.Dense(self.config['nbr_classes'], activation='sigmoid')(head_model)
        
        self.head_model = head_model
                
            
        # combine both models 
        self.model = tf.keras.Model(self.base_model.input, self.head_model)
        
    def weighted_categorical_crossentropy(self,weights):
        """
        A weighted version of keras.objectives.categorical_crossentropy

        Variables:
            weights: numpy array of shape (C,) where C is the number of classes

        Usage:
            weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
            loss = weighted_categorical_crossentropy(weights)
            model.compile(loss=loss,optimizer='adam')
        """

        weights = K.variable(weights)

        def loss(y_true, y_pred):
            # scale predictions so that the class probas of each sample sum to 1
            y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
            # clip to prevent NaN's and Inf's
            y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
            # calc
            loss = y_true * K.log(y_pred) * weights
            loss = -K.sum(loss, -1)
            return loss

        return loss
        
    def compile_model(self): 
        # optimizer
#         optimizer = tf.keras.optimizers.Adam(
#                                 learning_rate=self.config['train_parameters']['learning_rate'],
#                                 beta_1=0.9,
#                                 beta_2=0.999,
#                                 epsilon=1e-07,
#                                 amsgrad=False,
#                                 name="Adam",
#                             )
        optimizer = tf.keras.optimizers.SGD(
                learning_rate=self.config['train_parameters']['learning_rate'], momentum=0.9,
                nesterov=False, name="SGD"
            )

        # metric
        metrics = [tf.keras.metrics.CategoricalAccuracy(),
                  tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top 3 categorical acccuracy'), 
                  tf.keras.metrics.TopKCategoricalAccuracy(k=5, name='top 5 categorical acccuracy')
                  ]
    
    

#         loss = tf.keras.losses.CategoricalCrossentropy(
#                 from_logits=True,
#                 label_smoothing=0,
#                 reduction="auto",
#                 name="categorical_crossentropy_loss",
#              )
        #loss='sparse_categorical_crossentropy'
        #oss = 'categorical_crossentropy'
        
        #loss = self.weighted_categorical_crossentropy(np.ones(20)/20)
        
        #loss=tf.keras.losses.KLDivergence(reduction="auto", name="kl_divergence")
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0,name='binary_crossentropy')


        self.model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
        
    def train(self, name_run, notes, tags):
        # build generator and discriminator models 
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            try:
                # Currently, memory growth needs to be the same across GPUs
                for gpu in gpus:
                    tf.config.experimental.set_memory_growth(gpu, True)
                logical_gpus = tf.config.experimental.list_logical_devices('GPU')
                print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
            except RuntimeError as e:
                # Memory growth must be set before GPUs have been initialized
                print(e)
            
        # setup logging
        if self.config['logging_wandb']:
            # w&b 
            wandb.init(name=name_run, 
                   project=self.project_name,
                   notes=notes, 
                   tags=tags,
                   entity='cv-task-2')

            # save usefull config to w&b
            wandb.config.learning_rate = self.config['train_parameters']['learning_rate']
            wandb.config.batch_size = self.config['train_parameters']['batch_size']
            wandb.config.epochs = self.config['train_parameters']['epochs']
            wandb.config.steps_per_epoch = self.config['train_parameters']['steps_per_epoch']
            
            
        # build model 
        self.build()
        #self.model.summary()
        
        # set model parts trainable or not
        if self.config['train_base_model'] == False: 
            print('freezing base model layers')
            for layer in self.base_model.layers:
                layer.trainable = False
        if self.config['train_head_model'] == False: 
            print('freezing head model layers')
            for layer in self.head_model.layers:
                layer.trainable = False
        
        
        # compile model
        self.compile_model()
        
        if self.config['logging_wandb']:
            # set save_model true if you want wandb to upload weights once run has finished (takes some time)
            clbcks = [WandbCallback(save_model=False)]
        else: 
            clbcks = []

        
        # start training 
        history=self.model.fit(
                    x = self.dataset.train_generator(batch_size=self.config['train_parameters']['batch_size']),
                    steps_per_epoch = self.config['train_parameters']['steps_per_epoch'],
                    epochs=self.config['train_parameters']['epochs'], 
                    validation_data=self.dataset.validation_generator(batch_size=self.config['train_parameters']['batch_size']),
                    validation_steps=20, 
                    callbacks=clbcks
        )
        
        #workers=multiprocessing.cpu_count(),
        #use_multiprocessing=True,
        
        
        

        

## Training

In [ ]:
config = {
    'name': 'resnet50 KL divergence',
    'logging_wandb': False,  #nice tool for tracking a run. make and account on wandb.ai and I will add you to this project
    'weights': None, # 'imagenet', #None, 
    'nbr_classes': 20,
    'input_shape': (224, 224, 3),
    'train_base_model': True, # whether to train the head and or base model
    'train_head_model': True, 
    'train_parameters': {
        'epochs': 5,
        'batch_size': 64,
        'learning_rate': 0.001, 
        'steps_per_epoch': 1000
    },
    'dataset': {
        'train_fraction': 0.9,
        'input_shape': (224, 224),
        'augmentation': True, # whether to augment images or not
        'uniform_sample_probabilities': False
    },
    'head_model': {
        'head_model_units': [512, 512], 
        'add_dropout':      [False, False],
        'activation': 'relu'
    }
}

In [ ]:
RC = ResNetClassifactionModel(config)
RC.dataset.feed_preprocess_function(lambda x:x/255.0)
#RC.config = config
# RC.build()
# RC.model.summary()

In [ ]:
name_run='resnet_50 KL divergence'
notes='Binary crossentropy'
tags = ['resnet50', 'head = [1024, 1024]', 'head = [True, True]', 'Augmentation applied', 'loss: KLDivergence']
RC.set_config(config)
RC.train(name_run, notes, tags)


In [ ]:
RC.dataset.show_training_sampling_distribution()

In [ ]:
RC.dataset.show_class_distribution()

# Inference resnet50

In [ ]:
nbr=5
test_data=test_df["img"].to_numpy()
batch = test_data[nbr]
print(batch.shape)
# idx = np.random.randint(10,255, 3*500*300)
# batch = idx.reshape(300,500,3)
print(batch.shape)
plt.imshow(batch)

# RC.build()
# RC.model.load_weights('resnet50_heads_1024_1024.h5')

In [ ]:
RC.predict(batch)

# custom model

In [ ]:
class CustomModel(ResNetClassifactionModel): 
    def __init__(self, config):
        self.config = config 
        self.config_head = config['head_model']
        self.project_name = 'custom'
        
        # initialize dataset
        self.dataset = Dataset_Classification(config['dataset'])
        
        # feed preprocessor to dataset
        print('Feeding resnet50 preprocess function to dataset class')
        self.dataset.feed_preprocess_function(lambda x: x/255.0)
    
    def build(self):
        print('simple cnn')
        model = keras.models.Sequential([
            keras.layers.Conv2D(64,7, activation='relu', padding="same", 
                                input_shape=self.config['input_shape']), 
            keras.layers.MaxPooling2D(2), 
            keras.layers.Conv2D(128,3,activation='relu', padding='same'),
            keras.layers.Conv2D(128,3,activation='relu', padding='same'),
            keras.layers.MaxPooling2D(2),
            keras.layers.Conv2D(256,3,activation='relu', padding='same'),
            keras.layers.Conv2D(256,3,activation='relu', padding='same'),
            keras.layers.MaxPooling2D(2), 

            keras.layers.Flatten(), 
            keras.layers.Dense(128, activation='relu'), 
            keras.layers.Dropout(0.5), 
            keras.layers.Dense(64, activation='relu'), 
            keras.layers.Dropout(0.5), 
            keras.layers.Dense(self.config['nbr_classes'], activation='softmax')
            ])
        self.model=model

In [ ]:
config = {
    'name': 'resnet50 KL divergence',
    'logging_wandb': False,  #nice tool for tracking a run. make and account on wandb.ai and I will add you to this project
    'weights': 'imagenet', #None, 
    'nbr_classes': 20,
    'input_shape': (224, 224, 3),
    'train_base_model': True, # whether to train the head and or base model
    'train_head_model': True, 
    'train_parameters': {
        'epochs': 20,
        'batch_size': 64,
        'learning_rate': 0.01, 
        'steps_per_epoch': 100
    },
    'dataset': {
        'train_fraction': 0.9,
        'input_shape': (224, 224),
        'augmentation': True # whether to augment images or not
    },
    'head_model': {
        'head_model_units': [1024, 1024], 
        'add_dropout':      [True, True],
        'activation': 'relu'
    }
}

In [ ]:
Custom = CustomModel(config)

In [ ]:
name_run=''
notes=''
tags = ['custom']
Custom.train(name_run, notes, tags)

## VGG19

In [ ]:
class VGG19ClassifactionModel(ResNetClassifactionModel): 
    def __init__(self, config): 
        self.config = config 
        self.config_head = config['head_model']
        
        self.project_name = 'VGG19'
        
        # initialize dataset
        self.dataset = Dataset_Classification(config['dataset'])
        
        # feed preprocessor to dataset
        print('Feeding vgg19 preprocess function to dataset class')
        self.dataset.feed_preprocess_function(tf.keras.applications.vgg19.preprocess_input)
        
        # check if some configurations make sense 
        assert len(self.config_head['head_model_units']) == len(self.config_head['add_dropout']), 'head_models_units and add_dropout list should have same size'
        
    def build(self): 
        """
            Builds the model 
        """
        # define a resnet50 base model
        VGG19 = tf.keras.applications.VGG19(
                    include_top=False,
                    weights=self.config['weights'],
                    input_shape=self.config['input_shape'],
                        )
    
        self.base_model = VGG19
        
        # define a head model
        head_model=VGG19.output
        #head_model=tf.keras.layers.AveragePooling2D(pool_size=(7,7))(head_model)
        head_model=tf.keras.layers.Flatten()(head_model)
        
        for (nbr_units, dropout) in zip(self.config_head['head_model_units'], self.config_head['add_dropout']): 
            head_model=tf.keras.layers.Dense(nbr_units, activation=self.config_head['activation'])(head_model)
            if dropout:
                head_model=tf.keras.layers.Dropout(0.2)(head_model)
        
        head_model=tf.keras.layers.Dense(self.config['nbr_classes'], activation='linear')(head_model)
        
        self.head_model = head_model
                
            
        # combine both models 
        self.model = tf.keras.Model(self.base_model.input, self.head_model)

In [ ]:
config = {
    'name': 'vgg19',
    'logging_wandb': False, 
    'weights': None, 
    'nbr_classes': 20,
    'input_shape': (224, 224, 3),
    'train_base_model': True, # whether to train the head and or base model
    'train_head_model': True, 
    'train_parameters': {
        'epochs': 10,
        'batch_size': 32,
        'learning_rate': 0.001, 
        'steps_per_epoch': 100
    },
    'dataset': {
        'train_fraction': 0.9,
        'input_shape': (224, 224),
        'augmentation': True # whether to augment images or not
    },
    'head_model': {
        'head_model_units': [2048], 
        'add_dropout':      [False],
        'activation': 'linear'
    }
}
#64, 128, 256, 512, 1024, 2048, 4096

In [ ]:
vgg_model=VGG19ClassifactionModel(config)
# vgg_model.build()
# vgg_model.model.summary()

In [ ]:
name_run='vgg19_'
notes='First try for finetuning vgg19 on pretrained imagenet weights. Data augmentation turned on. '
tags = ['resnet50', 'head = [2048]', 'head = [False]', 'Augmentation applied']
vgg_model.train(name_run, notes, tags)


In [ ]:
dataset_config = {
    'train_fraction': 0.9,
    'input_shape': (224, 224),
    'augmentation': False
}
ds = Dataset_Segmentation(dataset_config)
# resnet50 preprocessor 
ds.feed_preprocess_function(lambda x: x/255.0)

i = 0
for X,y in ds.train_generator(0): 
    if i > 1: 
        break 
    i+=1 
print(X.shape)
print(y.shape)
fig, axes = plt.subplots(1,2)
axes[0].imshow(X[0])
axes[1].imshow(y[0])

In [ ]:
print(y[0].shape)
print(np.unique(y[0]))
idx = np.where(y[0] == 2)
zrs = np.zeros(y[0].shape)
zrs[idx]=1
plt.imshow(zrs)

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
model=tf.keras.applications.ResNet50V2()
preds = model.predict(im_pr)
tf.keras.applications.imagenet_utils.decode_predictions(
    preds, top=5
)


# 3. Semantic segmentation
The goal here is to implement a segmentation CNN that labels every pixel in the image as belonging to one of the 20 classes (and/or background). Use the training set to train your CNN and compete on the test set (by filling in the segmentation column in the test dataframe).

In [ ]:
class RandomSegmentationModel:
    """
    Random segmentation model: 
        - generates random label maps for the inputs based on the class distributions observed during training
        - every pixel in an input can only have one label
    """
    def fit(self, X, Y):
        """
        Adjusts the class ratio variable to the one observed in Y. 

        Parameters
        ----------
        X: list of arrays - n x (height x width x 3)
        Y: list of arrays - n x (height x width)

        Returns
        -------
        self
        """
        self.distribution = np.mean([[np.sum(Y_ == i) / Y_.size for i in range(len(labels) + 1)] for Y_ in Y], axis=0)
        print("Setting class distribution to:\nbackground: {}\n{}".format(self.distribution[0], "\n".join(f"{label}: {p}" for label, p in zip(labels, self.distribution[1:]))))
        return self
        
    def predict(self, X):
        """
        Predicts for each input a label map.
        
        Parameters
        ----------
        X: list of arrays - n x (height x width x 3)
             
        Returns
        -------
        Y_pred: list of arrays - n x (height x width)
        """
        np.random.seed(0)
        return [np.random.choice(np.arange(len(labels) + 1), size=X_.shape[:2], p=self.distribution) for X_ in X]
    
    def __call__(self, X):
        return self.predict(X)
    
model = RandomSegmentationModel()
model.fit(train_df["img"], train_df["seg"])
test_df.loc[:, "seg"] = model.predict(test_df["img"])
test_df.head(1)

In [ ]:
class UnetSegmentationMode(ResNetClassifactionModel): 
    def __init__(self, config): 
        self.config=config
        self.config_head = config['head_model']
        self.output_channels=self.config['nbr_classes'] + 1 # plus one since background is a class
        
        # initialize dataset
        self.dataset = Dataset_Segmentation(config['dataset'])
        
        # feed preprocessor to dataset
        print('Feeding vgg19 preprocess function to dataset class')
        self.dataset.feed_preprocess_function(tf.keras.applications.vgg19.preprocess_input)
        
        self.project_name = 'Unet'
        
        
    ## 
    def upsample(self, filters, size, apply_dropout=False):
        initializer = tf.random_normal_initializer(0., 0.02)

        result = tf.keras.Sequential()
        result.add(
        tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                        padding='same',
                                        kernel_initializer=initializer,
                                        use_bias=False))

        result.add(tf.keras.layers.BatchNormalization())

        if apply_dropout:
            result.add(tf.keras.layers.Dropout(0.5))

        result.add(tf.keras.layers.ReLU())

        return result
    
    def compile_model(self): 
        # optimizer
        optimizer = tf.keras.optimizers.Adam(
                                learning_rate=self.config['train_parameters']['learning_rate'],
                                beta_1=0.9,
                                beta_2=0.999,
                                epsilon=1e-07,
                                amsgrad=False,
                                name="Adam",
                            )

        metrics = ['acc', 
                  tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name='top 3 categorical acccuracy'), 
                  tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name='top 5 categorical acccuracy')
                  ]


        loss=loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


        self.model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
        
    def build(self):
        ## base model
        base_model = tf.keras.applications.MobileNetV2(input_shape=self.config['input_shape'], include_top=False)

        # Use the activations of these layers
        layer_names = [
            'block_1_expand_relu',   # 64x64
            'block_3_expand_relu',   # 32x32
            'block_6_expand_relu',   # 16x16
            'block_13_expand_relu',  # 8x8
            'block_16_project',      # 4x4
        ]
        base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

        # Create the feature extraction model
        down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs)

        down_stack.trainable = False # hard code it to be not trainable
        
        self.base_model = down_stack
        ## head model
        
        up_stack = [
            self.upsample(512, 3, self.config_head['add_dropout']),  # 4x4 -> 8x8
            self.upsample(256, 3, self.config_head['add_dropout']),  # 8x8 -> 16x16
            self.upsample(128, 3, self.config_head['add_dropout']),  # 16x16 -> 32x32
            self.upsample(64, 3, self.config_head['add_dropout']),   # 32x32 -> 64x64
        ]
        
        inputs = tf.keras.layers.Input(shape=self.config['input_shape'])

        # Downsampling through the model
        skips = down_stack(inputs)
        x = skips[-1]
        skips = reversed(skips[:-1])

        # Upsampling and establishing the skip connections
        for up, skip in zip(up_stack, skips):
            x = up(x)
            concat = tf.keras.layers.Concatenate()
            x = concat([x, skip])

        # This is the last layer of the model
        last = tf.keras.layers.Conv2DTranspose(
            self.output_channels, 3, strides=2,
            padding='same')  #64x64 -> 128x128

        x = last(x)

        self.model = tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
Unet_config = {
    'name': 'Unet',
    'logging_wandb': True, 
    'nbr_classes': 20,
    'train_base_model': False, # whether to train the head and or base model
    'train_head_model': True, # not defined here
    'input_shape': (224, 224, 3),
    'train_parameters': {
        'epochs': 10,
        'batch_size': 64,
        'learning_rate': 0.001, 
        'steps_per_epoch': 1000
    },
    'dataset': {
        'train_fraction': 0.9,
        'input_shape': (224, 224),
        'augmentation': True # whether to augment images or not
    },
    'head_model': {
        'add_dropout': True,
    }
}

In [ ]:
Unet = UnetSegmentationMode(Unet_config)

In [ ]:
name_run='Unet 1'
notes='Changed configurations, higher batch size, more steps per epoch'
tags = ['unet', 'Augmentation applied', 'dropout added']
Unet.train(name_run, notes, tags)

## Submit to competition
You don't need to edit this section. Just use it at the right position in the notebook. See the definition of this function in Sect. 1.3 for more details.

In [ ]:
generate_submission(test_df)

# 4. Adversarial attack
For this part, your goal is to fool your classification and/or segmentation CNN, using an *adversarial attack*. More specifically, the goal is build a CNN to perturb test images in a way that (i) they look unperturbed to humans; but (ii) the CNN classifies/segments these images in line with the perturbations.

# 5. Discussion
Finally, take some time to reflect on what you have learned during this assignment. Reflect and produce an overall discussion with links to the lectures and "real world" computer vision.